In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision

### L1 & L2 Regularization (Weight Decay)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = nn.Sequential(
    nn.Linear(784, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64,10)
).to(device)

optimizer = optim.Adam(model.parameters(), weight_decay=0.01)   # L2 Regularization
criterion = nn.CrossEntropyLoss()

# Load MNIST dataset (but take only 10 samples)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

full_train_dataset = torchvision.datasets.MNIST(root="./data", train=True, transform=transform, download=True)

# Select only the first 10 samples
small_train_dataset = torch.utils.data.Subset(full_train_dataset, range(10))
small_train_loader = torch.utils.data.DataLoader(small_train_dataset, batch_size=2, shuffle=True)


# Training loop (only 3 epochs for quick results)
epochs = 3
for epoch in range(epochs):
    for images, labels in small_train_loader:
        images = images.view(-1, 28*28).to(device)  # Flatten images
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# Test on the same 10 samples
correct = 0
total = 0
with torch.no_grad():
    for images, labels in small_train_loader:
        images = images.view(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)

        # Get predictions
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy on 10 Samples: {accuracy:.2f}%")


Epoch [1/3], Loss: 2.1749
Epoch [2/3], Loss: 2.3909
Epoch [3/3], Loss: 1.6803
Test Accuracy on 10 Samples: 50.00%


In [3]:
import torch.nn.functional as F

### Dropout Regularization

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)  # Apply dropout
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

model= Net().to(device)

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), weight_decay=0.01)  # L2 Regularization
criterion = nn.NLLLoss()  # Negative Log Likelihood Loss for classification

# Load MNIST dataset (but use only 10 samples for quick training)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
full_train_dataset = torchvision.datasets.MNIST(root="./data", train=True, transform=transform, download=True)

# Select only the first 10 samples
small_train_dataset = torch.utils.data.Subset(full_train_dataset, range(10))
small_train_loader = torch.utils.data.DataLoader(small_train_dataset, batch_size=2, shuffle=True)

# Training loop (only 3 epochs for quick results)
epochs = 3
for epoch in range(epochs):
    for images, labels in small_train_loader:
        images = images.view(-1, 28*28).to(device)  # Flatten images
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# Test on the same 10 samples
correct = 0
total = 0
with torch.no_grad():
    for images, labels in small_train_loader:
        images = images.view(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)

        # Get predictions
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy on 10 Samples: {accuracy:.2f}%")


Epoch [1/3], Loss: 2.9188
Epoch [2/3], Loss: 1.7094
Epoch [3/3], Loss: 2.1067
Test Accuracy on 10 Samples: 30.00%
